# Profiling Test

In [ ]:
%load_ext autoreload
%autoreload 2

from transformers import Qwen2VLForConditionalGeneration, AutoProcessor
from qwen_vl_utils import process_vision_info
import torch

# default processer
# processor = AutoProcessor.from_pretrained("Qwen/Qwen2-VL-7B-Instruct")

# The default range for the number of visual tokens per image in the model is 4-16384. You can set min_pixels and max_pixels according to your needs, such as a token count range of 256-1280, to balance speed and memory usage.
min_pixels = 256*28*28
max_pixels = 1280*28*28
processor = AutoProcessor.from_pretrained("Qwen/Qwen2-VL-7B-Instruct", min_pixels=min_pixels, max_pixels=max_pixels)

messages = [
    {
        "role": "user",
        "content": [
            {
                "type": "image",
                "image": "https://qianwen-res.oss-cn-beijing.aliyuncs.com/Qwen-VL/assets/demo.jpeg",
            },
            {"type": "text", "text": "Describe this image."},
        ],
    }
]

# Preparation for inference
text = processor.apply_chat_template(
    messages, tokenize=False, add_generation_prompt=True
)
image_inputs, video_inputs = process_vision_info(messages)
inputs = processor(
    text=[text],
    images=image_inputs,
    videos=video_inputs,
    padding=True,
    return_tensors="pt",
)
inputs = inputs.to("cuda")

# default: Load the model on the available device(s)
model = Qwen2VLForConditionalGeneration.from_pretrained(
    "Qwen/Qwen2-VL-7B-Instruct", torch_dtype="auto", device_map="auto"
)

# Inference: Generation of the output
generated_ids = model.generate(**inputs, max_new_tokens=128)
model.print_time()
generated_ids_trimmed = [
    out_ids[len(in_ids) :] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
]
output_text = processor.batch_decode(
    generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False
)
print(output_text)

# Baseline Evaluation

In [ ]:
%load_ext autoreload
%autoreload 2

import torch
from transformers import Qwen2VLConfig, Qwen2VLForConditionalGeneration, TrainingArguments, Trainer, AutoProcessor
import evaluate
import numpy as np
from datasets import load_dataset
import time

import baseline
import efficient
from utils import get_dataset, train

Make dataset

In [ ]:
min_pixels = 16*28*28
max_pixels = 80*28*28
processor = AutoProcessor.from_pretrained("Qwen/Qwen2-VL-7B-Instruct", min_pixels=min_pixels, max_pixels=max_pixels)

messages = [{
    "role": "user",
    "content": [
        {"type": "image"},
    ],
}]

text = processor.apply_chat_template(
    messages, tokenize=False, add_generation_prompt=True
)

datasets = load_dataset("mnist", cache_dir='.cache')
train_dataset = datasets["train"].select(range(1000))
test_dataset = datasets["test"].select(range(100))

def process_function(datum):
    return processor(text=text, images=datum["image"], videos=None, padding=True, return_tensors="pt")

train_dataset = train_dataset.map(process_function, batched=False).shuffle(seed=1512)
test_dataset = test_dataset.map(process_function, batched=False).shuffle(seed=1512)

Train and evaluate baseline model

In [ ]:
baseline_config = Qwen2VLConfig(
    hidden_size=16,
    intermediate_size=128,
    num_hidden_layers=3,
    num_attention_heads=8,
    rope_scaling={"mrope_section": [1, 1, 2], "type": "mrope"},
    vision_config={"hidden_size": 64,
                   "in_channels": 3,
                   "embed_dim": 64,
                   "num_heads": 1,
                   "spatial_merge_size": 2,
                   "patch_size": 14,
                   "temporal_patch_size": 2,
                   "depth": 3,
                   "vocab_size": 10,
                   "spatial_merge_size": 2},
    _name_or_path="baseline",
    image_token_id=151655,
)

#print(baseline_config)

baseline_model = baseline.Qwen2VLBaseline(baseline_config.vision_config)

In [ ]:
print(train_dataset)

In [ ]:
class Args:
    device = "cpu"
    lr_net = 0.005
    epoch_eval_train = 10

args = Args()

In [ ]:
train(0, baseline_model, train_dataset, test_dataset, args)

Train and evaluate efficient model

In [ ]:
efficient_config = Qwen2VLConfig(
    hidden_size=16,
    intermediate_size=128,
    num_hidden_layers=3,
    num_attention_heads=8,
    rope_scaling={"mrope_section": [1, 1, 2], "type": "mrope"},
    vision_config={"hidden_size": 64,
                   "in_channels": 3,
                   "embed_dim": 64,
                   "num_heads": 1,
                   "spatial_merge_size": 2,
                   "patch_size": 14,
                   "temporal_patch_size": 2,
                   "depth": 3,
                   "vocab_size": 10,
                   "spatial_merge_size": 2},
    _name_or_path="efficient",
    image_token_id=151655,
)

efficient_model = efficient.Qwen2VLEfficient(efficient_config.vision_config)

In [ ]:
class Args:
    device = "cuda"
    lr_net = 0.005
    epoch_eval_train = 10

args = Args()

In [ ]:
train(0, efficient_model, train_dataset, test_dataset, args)

In [ ]:
x = torch.randn([64,64]).numpy()
np.sum(np.abs(x), 0)

In [ ]:
most_impactful = np.argpartition(np.sum(np.abs(x), 0), 32)[32:]
x[most_impactful]